Para poder visualizar los árboles de decisión que entrenaremos necesitamos instalar una librearía específica

In [ ]:
conda install python-graphviz

# Overfitting y Underfitting

Como vimos, un modelo sobreajustado es un modelo que es tan específico en su entrenamiento que analiza a la perfeccción los datos conocidos pero al darle datos nuevos no lográ hacer una buena predicción. El caso de un modelo subajustado es el contrario, cuando es demasiado genérico en su predicción y no logra los resultados esperados. 

En esta Notebook veremos un ejemplo de un modelo sobreajustado y forma de resolverlo. El ejemplo que utilizaremos será un Árbol de decisión que es un modelo que tiende al sobreajuste.

Vamos a realizar los pasos de un Proyecto de Ciencia de datos e intentaremos mejorar su performance. 

1. Definición del Problema
2. Búsqueda de datos 
3. Exploración y Limpieza de Datos
4. Dividir los datos en **X** (variables predictoras) e **y** (variable a predecir). Dividir los datos en entrenamiento y testo con el méodo *train_test_split*
5. Entrenamiento del modelo
6. Testeo del Modelo 

### Definición del problema

**¿Es peligroso un asteroide cercano a la Tierra en base a determinadas características?**

### Búsqueda de datos

El dataset que se utilizará es de la Nasa y muestra distintos asteroides cercanos a la Tierra y sus características para poder predecir su peligrosidad *hazardous (peligroso)*. Fue descargado de [Kaggle](https://www.kaggle.com/datasets/sameepvani/nasa-nearest-earth-objects)



In [ ]:
#importamos las librerías que utilizaremos

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("neo.csv")

##### Exploración y limpieza del dataset

In [ ]:
data.head(3)

In [ ]:
data.info()

In [ ]:
data["orbiting_body"].value_counts()

##### Limpieza del dataset

Eliminamos las variables *id* y *name* ya que no servirán para el entrenamiento del modelo. 
Luego de realizado esto no habrá variables categóricas (salvo la variable a predecir).
También eliminamos la variable *orbiting_body* ya que en todos los casos recibe el mismo valor por lo cual tampoco es útil para el entrenamiento del modelo. 

Luego observamos la distribución de la variable target *hazardous*

In [ ]:
# Eliminamos las variables que no serán útiles para el entrenamiento del modelo

data.drop(columns= ["id", "name", "orbiting_body"], inplace=True)
data.head(3)

In [ ]:
#Distribución de la variable target

data["hazardous"].value_counts()

In [ ]:
sns.countplot(data["hazardous"])

#### Entrenamiento del modelo

Vamos a entrenar un modelo de Árbol de decisión para la peligrosidad de un asteroide. En primer lugar dividermos los datos en **X** e **y** y en entrenamiento y testo.

Instanciaremos el modelo de Árbol de decisión con los hiperparámentros por default.

In [ ]:
# Generamos X e y 

X = data.drop(columns = "hazardous")   #variables predictora
y = data["hazardous"]   #variable a predecir

In [ ]:
# Dividimos datos en train y test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=138)  #por default 25% de test


In [ ]:
# Importamos el modelo que utilizaremos

from sklearn.tree import DecisionTreeClassifier


In [ ]:
#Instanciamos el modelo que utilizaremos

arbol = DecisionTreeClassifier()

In [ ]:
#Entrenamos el modelo 

arbol.fit(X_train, y_train)

##### Visualización del Árbol generado por el modelo

Dada la cantidad de valores que tiene nuestro dataset y lo específico del arbol, al intentar graficarlo vamos a tener problemas de performance en nuestra notebook. 

Podemos conocer la cantidad de nodos que tiene nuestro Árbol

In [ ]:
arbol.tree_.node_count

In [ ]:
arbol.tree_.max_depth

##### Evaluación de datos de train

En este caso, además de utilizar las métricas para evaluar los datos de testo, también utilizaremos las métricas para evaluar cómo el modelo fue entrenado, es decir, si utilizamos el método predict con los datos de entrenamiento y comparamos con los datos de entrenamiento. 

Esto no sirve para saber la performance del modelo ya que fue entrenado con los mismo datos, pero sirve para analizar como fue entrenado el modelo y, en el caso de los árboles de decisión, la exactitud con la que los datos se organizan.

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_train = arbol.predict(X_train)
exactitud_train_arbol = accuracy_score(y_train, y_pred_train)
exactitud_train_arbol

Podemos observar que el resultado de evaluar los datos de entrenamiento es perfecta

##### Probamos y evaluamos nuestro modelo

Utilizamos el metodo *predict* para probar nuestro modelo con los datos de test. Luego comparamos la predicciones de nuestro modelo con el resultado real a través de una matrix de confusión y utilizando la métrica accuracy (exactitud)

In [ ]:
# Probamos nuestro modelo con los datos de test

y_pred_arbol = arbol.predict(X_test)


In [ ]:
# Matriz de confusión:comparando resultado original (y_test) con predicción del modelo (y_pred_arbol_completo)
from sklearn.metrics import confusion_matrix

matriz_arbol = confusion_matrix(y_test, y_pred_arbol)
sns.heatmap(matriz_arbol, annot=True, fmt = "d")
plt.xlabel("Etiquetas predichas")
plt.ylabel("Etiquetas reales")

In [ ]:
#metrica accuracy

exactitud_arbol = accuracy_score(y_test, y_pred_arbol)
exactitud_arbol

In [ ]:
print("La exactitud del modelo árbol de decisión es", round(exactitud_arbol,2))

#### Hiperparámetro distintos

Como hablamso los árboles de decisión, estos tienen una tendencia al sobreajuste (overfitting). 
Para evitar eso es posible ajustar los distintos hiperpárametros para reducir la complejidad de los árboles, se utiliza un mecanismo denominado "poda", reduciendo el tamaño del árbol a través de limitar la profundidad máxima, limitar el número de muestrar requeridas por cada hoja o limitando el número mínimo de muestras para particionar. Se puede ver en la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html?highlight=tree#sklearn.tree.DecisionTreeClassifier)

Algunos de los hiperparámetros que se pueden ajustar en el modelo de Scikit-Lear son:.

- **max_depth:** profundidad máxima del árbol. Solemos determinar una profundidad máxima para evitar que el modelo sobreajuste.  
- **min_samples_split:** número mínimo de muestras que un nodo debe contener para considerar la división. El valor predeterminado es dos. Podemos usar este parámetro para regularizar el árbol.
- **min_samples_leaf:** número mínimo de muestras necesarias para ser considerado un nodo hoja. El valor predeterminado se establece en uno. Este parámetro se utiliza como una forma alternativa de limitar el crecimiento del árbol.  
- **max_features:** número de características a considerar al buscar la mejor división. Si no se establece este valor, el árbol de decisión considerará todas las variables independientes disponibles para hacer la mejor división.

En este caso vamos a entrenar y evaluar dos modelos modificando el hiperparámetro: "max_depth" y observando la diferencia a través de las visualizaciones

In [ ]:
# Instanciamos el modelo que utilizaremos

arbol_depth4 = DecisionTreeClassifier(max_depth=4)

In [ ]:
# Entrenamos el modelo 

arbol_depth4.fit(X_train, y_train)

In [ ]:
from sklearn import tree
class_names = ['True', 'False']
plt.figure(figsize = (20,20))
tree.plot_tree(arbol_depth4, feature_names=data.columns[:-1],filled=True,rounded=True, class_names=class_names)


##### Evaluación de datos de train

Podemos utilizar una métrica de evaluación para ver la performance del modelo con los datos de entrenamiento

In [ ]:
y_pred_train_depth4 = arbol_depth4.predict(X_train)
exactitud_train_depth4 = accuracy_score(y_train, y_pred_train_depth4)
exactitud_train_depth4

##### Probamos y evaluamos nuestro modelo

Utilizamos el metodo predict para probar nuestro modelo con los datos de test. Luego comparamos la predicciones de nuestro modelo con el resultado real a través de una matrix de confusión y utilizando la métrica accuracy (exactitud)

In [ ]:
# Probar nuestro modelo con los datos de test

y_pred_depth4 = arbol_depth4.predict(X_test)
y_pred_depth4

In [ ]:
# Matriz de confusión:comparando resultado original (y_test) con predicción del modelo (y_pred_depth4)
matriz_depth4 = confusion_matrix(y_test, y_pred_depth4)
sns.heatmap(matriz_depth4, annot=True, fmt="d")
plt.xlabel("Etiquetas predichas")
plt.ylabel("Etiquetas reales")

In [ ]:
exactitud_depth4 = accuracy_score(y_test, y_pred_depth4)
exactitud_depth4

In [ ]:
print("La exactitud del modelo con los hiperparámetros por default es", round(exactitud_arbol,2))
print("La exactitud del modelo hiperparámetro profundidad máxima de 4 es", round(exactitud_depth4,2))

#### Entrenamos nuestro modelo con otro hiperparámetro

In [ ]:
# Instanciamos el modelo que utilizaremos

arbol_depth2 = DecisionTreeClassifier(max_depth=2)

In [ ]:
# Entrenamos el modelo 

arbol_depth2.fit(X_train, y_train)

In [ ]:
class_names = ['True', 'False']
plt.figure(figsize = (10,10))
tree.plot_tree(arbol_depth2, feature_names=data.columns[:-1],filled=True,rounded=True, class_names=class_names)


##### Evaluación de datos de train

Podemos utilizar una métrica de evaluación para ver la performance del modelo con los datos de entrenamiento

In [ ]:
y_pred_train_depth2 = arbol_depth2.predict(X_train)
exactitud_train_depth2 = accuracy_score(y_train, y_pred_train_depth2)
exactitud_train_depth2

##### Probamos y evaluamos nuestro modelo

Utilizamos el metodo predict para probar nuestro modelo con los datos de test. Luego comparamos la predicciones de nuestro modelo con el resultado real a través de una matrix de confusión y utilizando la métrica accuracy (exactitud)

In [ ]:
# Probar nuestro modelo con los datos de test

y_pred_depth2 = arbol_depth2.predict(X_test)


In [ ]:
# Matriz de confusión:comparando resultado original (y_test) con predicción del modelo (y_pred_depth2)
matriz_depth2 = confusion_matrix(y_test, y_pred_depth2)
sns.heatmap(matriz_depth2, annot=True, fmt="d")
plt.xlabel("Etiquetas predichas")
plt.ylabel("Etiquetas reales")

In [ ]:
#accuracy

exactitud_depth2 = accuracy_score(y_test, y_pred_depth2)
exactitud_depth2

In [ ]:
#Comparamos todos los modelos

print("La exactitud del modelo con los hiperparámetros por default es", round(exactitud_arbol,2))
print("La exactitud del modelo con hiperparámetro profundidad máxima de 4 es", round(exactitud_depth4,2))
print("La exactitud del modelo con hiperparámetro profundidad máxima de 2 es", round(exactitud_depth2,2))


### Conclusión

El Árbol de decisión entrenado tiende al sobreajuste. Esto significa que es tan correcto el entrenamiento (la performance en los datos de entrenamiento) pero puede ser malo al darle datos nuevo como los datos de testeo (y en la realidad), ya que es demasiado específico para los datos con los que fue entrenado.

En este ejemplo vimos como "recortando" el árbol y simplificandolo empeoró la performance para los datos de entrenamiento pero mejoró para los datos de testeo. 

